In [1]:
# Import libraries and dependencies
import ccxt
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import talib as ta
import yfinance as yf 

from Utility_Functions import Functions

# Downloading the data from yfinance

### Set the currencies, date range and the interval for which you want the data

In [2]:
currs_list = [ 'BTC-AUD', 'ETH-AUD', 'XRP-AUD' , 'LTC-AUD', 'ADA-AUD', 'XLM-AUD', 'BCH-AUD']     #
start_date = '2019-06-01'
end_date = '2021-07-01'
interval = '1h'

### Call yfinance api

In [3]:
df_data = yf.download(currs_list, start= start_date, end= end_date, interval= interval, group_by= 'ticker')
df_data.head(2)


[*********************100%***********************]  7 of 7 completed


XRP-AUD                                              \
                             Open    High     Low   Close Adj Close    Volume   
2019-09-18 00:00:00+01:00  0.4223  0.4252  0.4170  0.4200    0.4200       0.0   
2019-09-18 01:00:00+01:00  0.4180  0.4419  0.4176  0.4419    0.4419  121342.0   

                              ETH-AUD                                      \
                                 Open        High         Low       Close   
2019-09-18 00:00:00+01:00  308.279999  309.230011  304.089996  304.089996   
2019-09-18 01:00:00+01:00  304.440002  309.279999  302.350006  309.279999   

                           ...   XLM-AUD                             \
                           ...       Low     Close Adj Close Volume   
2019-09-18 00:00:00+01:00  ...  0.092567  0.092567  0.092567    0.0   
2019-09-18 01:00:00+01:00  ...  0.092567  0.095366  0.095366    0.0   

                              LTC-AUD                                      \
                                 Open        High         Low       Close   
2019-09-18 00:00:00+01:00  109.739998  110.489998  108.330002  108.510002   
2019-09-18 01:00:00+01:00  108.489998  109.120003  108.410004  109.120003   

                                               
                            Adj Close  Volume  
2019-09-18 00:00:00+01:00  108.510002     0.0  
2019-09-18 01:00:00+01:00  109.120003  7942.0  

[2 rows x 42 columns]

### Split the data by currency and add it to the dictionary

Dictionary Structure: 
* Key= currency code 
* value = Dataframe conatining OHLCV data

In [4]:
df_data.shape

(15603, 42)

In [5]:
dict_ohlcv = {}

for curr in currs_list:

    df_ohlcv = df_data.loc[ :, ([curr]) ]
    curr_code = curr.replace('-', '/')

    df_ohlcv.columns = df_ohlcv.columns.droplevel(0)

    start_date = df_ohlcv.index[0].date().isoformat()
    end_date = df_ohlcv.index[-1].date().isoformat()
    num_records = (len(df_ohlcv))
    start_price = df_ohlcv.iloc[0]['Close']
    end_price = df_ohlcv.iloc[-1]['Close']

    print(f'Data summary for {curr_code}')
    print(f'    Start Date: {start_date}; End Date: {end_date}; NUmber of records: {num_records}')
    print(f'    Start Price: {start_price}; End Price: {end_price}')

    df_ohlcv.dropna(inplace=True)

    # Store the symbol name and history data in a dict 
    dict_ohlcv[curr_code] = df_ohlcv 

    print(f'Data for {curr_code} fetched and appended into the dictionary\n')


Data summary for BTC/AUD
    Start Date: 2019-09-18; End Date: 2021-06-30; NUmber of records: 15603
    Start Price: 14858.330078125; End Price: 46277.2109375
Data for BTC/AUD fetched and appended into the dictionary

Data summary for ETH/AUD
    Start Date: 2019-09-18; End Date: 2021-06-30; NUmber of records: 15603
    Start Price: 304.0899963378906; End Price: 2853.845458984375
Data for ETH/AUD fetched and appended into the dictionary

Data summary for XRP/AUD
    Start Date: 2019-09-18; End Date: 2021-06-30; NUmber of records: 15603
    Start Price: 0.41999998688697815; End Price: 0.8943524360656738
Data for XRP/AUD fetched and appended into the dictionary

Data summary for LTC/AUD
    Start Date: 2019-09-18; End Date: 2021-06-30; NUmber of records: 15603
    Start Price: 108.51000213623047; End Price: 186.65286254882812
Data for LTC/AUD fetched and appended into the dictionary

Data summary for ADA/AUD
    Start Date: 2019-09-18; End Date: 2021-06-30; NUmber of records: 15603
    S

# Compute the Technical Indicators using

In [6]:
fast_window = 5
slow_window = 15

df_all_data = pd.DataFrame() 
for curr, data in dict_ohlcv.items():

    df = Functions.add_tech_indicators(data, fast_window, slow_window)
    
    df['Currency'] = curr 
    df['Returns'] = df['Close'].pct_change()

    df.dropna(inplace=True)    
    df_all_data = df_all_data.append(df)

df_all_data.shape

(107683, 18)

In [7]:
df_all_data.to_csv('Resources/Training_data.csv', index=True)